In [0]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

In [11]:
!wget -O train.csv https://raw.githubusercontent.com/pcsanwald/kaggle-titanic/master/train.csv
!wget -O test.csv https://raw.githubusercontent.com/pcsanwald/kaggle-titanic/master/test.csv

--2019-03-01 08:37:02--  https://raw.githubusercontent.com/pcsanwald/kaggle-titanic/master/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57726 (56K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  56.37K  --.-KB/s    in 0.01s   

2019-03-01 08:37:02 (3.70 MB/s) - ‘train.csv’ saved [57726/57726]

--2019-03-01 08:37:03--  https://raw.githubusercontent.com/pcsanwald/kaggle-titanic/master/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26635 (26K) [text/plain]
Saving to: ‘test.csv’

test

In [44]:
train = pd.read_csv('train.csv', error_bad_lines=False)
#test = pd.read_csv('test.csv', error_bad_lines=False)
train.shape

(891, 11)

In [13]:
train.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [58]:
y = train.survived
X = train[['pclass', 'sex', 'age', 'sibsp', 'fare', 'parch']]
X = pd.get_dummies(X)
X.columns
y.value_counts()
X.apply(lambda x: sum(pd.isna(x)))
X[X.age.isna()] = X.age.median()
X.head()

,pclass,age,sibsp,fare,parch,sex_female,sex_male
0,3.0,22.0,1.0,7.2500,0.0,0.0,1.0
1,1.0,38.0,1.0,71.2833,0.0,1.0,0.0
2,3.0,26.0,0.0,7.9250,0.0,1.0,0.0
3,1.0,35.0,1.0,53.1000,0.0,1.0,0.0
4,3.0,35.0,0.0,8.0500,0.0,0.0,1.0


In [0]:
param_grid = {'n_estimators': range(50,250, 500),
              'max_features': range(1,8)}

In [60]:
rf = RandomForestClassifier()
best_model = GridSearchCV(estimator=rf,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=5,
                          refit=True,
                          verbose=1)
best_model.fit(X,y)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    2.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': range(50, 250, 500), 'max_features': range(1, 8)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [61]:
X.head()

,pclass,age,sibsp,fare,parch,sex_female,sex_male
0,3.0,22.0,1.0,7.2500,0.0,0.0,1.0
1,1.0,38.0,1.0,71.2833,0.0,1.0,0.0
2,3.0,26.0,0.0,7.9250,0.0,1.0,0.0
3,1.0,35.0,1.0,53.1000,0.0,1.0,0.0
4,3.0,35.0,0.0,8.0500,0.0,0.0,1.0


In [64]:
X_boy = pd.DataFrame({'pclass':[3], 'age':[2], 'sibsp':[3],'fare':[10], 'parch':[1], 'sex_female':[0], 'sex_male':[1]})
best_model.predict_proba(X_boy)

array([[0.36, 0.64]])